# Resolução da lista 3 de NLP
## Alunos:
    - Eduardo Brasil Araujo
    - Gideão Pinheiro

# Questão 1

# Questão 2

# Questão 3

In [33]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random

import pandas as pd

In [6]:
lstm_data = pd.read_csv('../datasets/preprocessed_website_classification.csv')
lstm_raw_data = lstm_data.cleaned_website_text

In [31]:
corpus = ''
for data in lstm_raw_data:
    corpus += f'{data} '

print('Corpus length:', len(corpus))

Corpus length: 6686147


In [32]:
chars = sorted(list(set(corpus)))
print('Total chars:', len(chars))

Total chars: 27


In [64]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [102]:
sentences = []
next_chars = []
Q = 10
for i in range(0, len(corpus) - Q, 8):
    sentences.append([char_indices[char] for char in corpus[i : i + Q]])
    next_chars.append(char_indices[corpus[i + Q]])

print('Number of sequences:', len(sentences))

Number of sequences: 835768


In [103]:
x = np.zeros((len(sentences), Q, len(chars)), dtype=bool)
y = np.zeros((len(sentences), len(chars)), dtype=bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char] = 1
    y[i, next_chars[i]] = 1

In [116]:
x = torch.tensor(x, dtype=torch.float32, device=torch.device('cuda'))
y = torch.tensor(y, dtype=torch.float32, device=torch.device('cuda'))

C:\Users\localhost\AppData\Local\Temp\ipykernel_18028\1269519880.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(x, dtype=torch.float32, device=torch.device('cuda'))
C:\Users\localhost\AppData\Local\Temp\ipykernel_18028\1269519880.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32, device=torch.device('cuda'))


In [119]:
print(x.shape)

torch.Size([835768, 10, 27])


In [109]:
class LSTMTextGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMTextGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq, hidden_state):
        output, hidden_state = self.lstm(input_seq, hidden_state)
        output = self.fc(output.view(input_seq.size(0), -1))
        return output, hidden_state

In [105]:
num_chars = len(chars)
print(num_chars)
input_size = num_chars
hidden_size = 128
output_size = num_chars
learning_rate = 0.01
num_epochs = 30

27


In [117]:
model = LSTMTextGenerator(input_size, hidden_size, output_size)
model.to(torch.device('cuda'))
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [118]:
for epoch in range(num_epochs):
    hidden_state = None
    loss = 0
    for seq, target in zip(x, y):
        optimizer.zero_grad()
        # seq = seq.view(Q, -1)
        output, hidden_state = model(seq, hidden_state)
        target = target.view(1)
        loss += criterion(output, target)
        print('Ended thing')
    loss.backward()
    optimizer.step()
    # if (epoch + 1) % 1 == 0:
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print('Treinamento concluído!')

RuntimeError: shape '[1]' is invalid for input of size 27

# Questão 4